# Particle trajectories in ocean current animated gif tutorial
This brief tutorial shows how to recreate the [animated gif](http://oceanparcels.org/animated-gifs/globcurrent_fullyseeded.gif) showing particles in the Agulhas region south of Africa.  The first half of this demo is based on the work of [Delandmeter and Van Sebille (2019)](https://www.geosci-model-dev.net/12/3571/2019/gmd-12-3571-2019.html) and [Ocean Parcels](https://oceanparcels.org/) and copied from this [Ocean Parcels example](https://github.com/OceanParcels/parcels/blob/master/parcels/examples/tutorial_Agulhasparticles.ipynb).  The second half of this notebook parallelizes this process with the [Parsl parallel Python scripting package](https://parsl-project.org).

## Local calculations

In [ ]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4, ErrorCode
from datetime import timedelta
import numpy as np

#=====================================
# Load data
#=====================================
filenames = "/pw/workflows/ocean_parcels_demo/parcels_examples/GlobCurrent_example_data/20*.nc"
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

#=====================================
# Define a lon, lat grid
#=====================================
lons, lats = np.meshgrid(range(15, 35), range(-40, -30))
pset = ParticleSet(fieldset=fieldset, pclass=JITParticle, lon=lons, lat=lats)

#=====================================
# Define particle boundary conditions
#=====================================
def DeleteParticle(particle, fieldset, time):
    particle.delete()

#=====================================
# Calculate particle positions
#=====================================
for cnt in range(3):
    # Set filename for output plot
    output_image = 'particles'+str(cnt).zfill(2)
    print(output_image)
    
    # First plot the particles
    pset.show(savefile=output_image, field='vector', land=True, vmax=2.0)

    # Then advect the particles for 6 hours
    pset.execute(AdvectionRK4,
                 runtime=timedelta(hours=6),  # runtime controls the interval of the plots
                 dt=timedelta(minutes=5),
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})  # the recovery kernel

# Execution in parallel on remote, custom workers

+ Run OceanParcels in separate experiments at the same time.
+ Each experiment is defined by a Python/OceanParsels script.
+ One experiment per (customizable) worker node.
+ Each instance of OceanParcels is run via a Docker container.

This calculation is launched right here from the notebook. Please ensure that a compute resource is turned on first and `pw.conf` is present in the directory that this notebook is running from.

In [ ]:
import parsl
from parsl.app.app import python_app, bash_app
from parsl.data_provider.files import File
from path import Path
from parsl.configs.local_threads import config
from parslpw import pwconfig,pwargs
parsl.load(pwconfig)

print("pwconfig loaded")

In [ ]:
@bash_app
def run_ocean_parcels(inputs=[], outputs=[]):
    
    import os
    config = os.path.basename(inputs[0])
    launcher = os.path.basename(inputs[1])
    out_file = os.path.splitext(os.path.basename(inputs[0]))[0]+'.gif'
    out_dir = os.path.basename(outputs[0])
    
    run_command = "/bin/bash " + launcher + " python " + config
    
    return '''
        %s
        outdir=%s
        outfile=%s
        mkdir -p $outdir
        mv movie.gif $outdir/$outfile
    ''' % (run_command,out_dir,out_file)

In [ ]:
configs = [Path('test_ocean_parcels_1.py'),Path('test_ocean_parcels_2.py')]
launcher = Path('wrap_docker.sh')
out_dir = Path('/pw/storage/PyData_Chicago')

app_futures=[]
for config in configs:
    
    future = run_ocean_parcels(inputs=[config,launcher], 
                          outputs=[out_dir])
    
    app_futures.append(future)

print("Running",len(app_futures),"OceanParcels executions...")
[f.result() for f in app_futures]